In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/profiles.csv")

First, let's drop all the columns we won't use.

In [3]:
df.drop(['last_online'], axis=1, inplace=True)
df.drop([f"essay{n}" for n in np.arange(0,10)], axis=1, inplace=True)

The "diet" variable has 6 possible cuisines (anything, vegetarian, vegan, kosher, halal, and other) and 2 changers (mostly/strictly), which might create unnecessary dummy variables that won't add value to the model. So let's boil it down to diet type without the nuance.

In [4]:
df['diet'].replace(["mostly ", "strictly "], "", regex=True, inplace=True)
df['diet']

0          anything
1             other
2          anything
3        vegetarian
4               NaN
            ...    
59941           NaN
59942      anything
59943      anything
59944      anything
59945           NaN
Name: diet, Length: 59946, dtype: object

The same goes for the `language` variable. There are 3 modifiers for a multitude of languages. The permutation between all possible values will generate thousands of unnecessary columns. Let's just keep the languages.

In [5]:
df['speaks'].replace([" \(fluently\)", " \(okay\)", " \(poorly\)"], "", regex=True, inplace=True)
df["speaks"]

0                                            english
1                           english, spanish, french
2                               english, french, c++
3                                    english, german
4                                            english
                            ...                     
59941                                        english
59942                                        english
59943                                        english
59944    english, spanish, chinese, korean, japanese
59945                                        english
Name: speaks, Length: 59946, dtype: object

But filtering out the language modifiers isn't enough. There's only one column for all the user's spoken languages, whose combination might generate a dozen thousands dummy variables. Therefore, each `speaks` value should be parsed and gain a dummy variable of its own.

In [6]:
def convert_speak_col(df):
    lang_spoken = df['speaks'].str.split(pat=", ", expand=True)
    # see https://stackoverflow.com/questions/26977076/pandas-unique-values-multiple-columns 
    lang_columns = pd.unique(lang_spoken.values.ravel('K'))
    table_shape = (df.shape[0], len(lang_columns))
    lang_table = pd.DataFrame(np.zeros(table_shape, dtype="int"),
                              columns=[f"speaks_{lang}" for lang in lang_columns])
    for index,row in lang_spoken.iterrows():
        for item in row:
            lang_table.at[index, f"speaks_{item}"] = 1
   
    lang_table.drop(["speaks_None"], axis=1, inplace=True)
    df_no_speak = df.drop("speaks", axis=1)
    return pd.concat([df_no_speak, lang_table], axis=1)

df_spoken_langs = convert_speak_col(df)

In [7]:
def create_dummy(df):
    """
    Creates dummy variables for all the categorial variables in a DataFrame and concatenates it
    with the original numerical columns.
    Input: pandas DataFrame
    Output: pandas DataFrame
    """
    cat_cols = df.select_dtypes(include="object").columns
    num_cols = df.select_dtypes(exclude="object").columns
    dummy_df = pd.get_dummies(df[cat_cols],
                              prefix=cat_cols,
                              prefix_sep="_",
                              dummy_na=True,
                              columns=cat_cols)
                              
    df_new = pd.concat([df[num_cols], dummy_df], axis=1)
    return df_new

In [8]:
df_dummies = create_dummy(df_spoken_langs)
df_dummies.head()

,age,height,income,speaks_english,speaks_nan,speaks_afrikaans,speaks_french,speaks_portuguese,speaks_spanish,speaks_german,...,smokes_trying to quit,smokes_when drinking,smokes_yes,smokes_nan,status_available,status_married,status_seeing someone,status_single,status_unknown,status_nan
0,22,75.0,-1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,35,70.0,80000,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,38,68.0,-1,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,23,71.0,20000,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,29,66.0,-1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
print([x for x in df_dummies.columns])

['age', 'height', 'income', 'speaks_english', 'speaks_nan', 'speaks_afrikaans', 'speaks_french', 'speaks_portuguese', 'speaks_spanish', 'speaks_german', 'speaks_chinese', 'speaks_sign language', 'speaks_c++', 'speaks_tagalog', 'speaks_other', 'speaks_russian', 'speaks_dutch', 'speaks_indonesian', 'speaks_swedish', 'speaks_belarusan', 'speaks_japanese', 'speaks_farsi', 'speaks_italian', 'speaks_hindi', 'speaks_polish', 'speaks_korean', 'speaks_czech', 'speaks_croatian', 'speaks_vietnamese', 'speaks_esperanto', 'speaks_latin', 'speaks_greek', 'speaks_norwegian', 'speaks_hebrew', 'speaks_arabic', 'speaks_tibetan', 'speaks_georgian', 'speaks_thai', 'speaks_swahili', 'speaks_khmer', 'speaks_turkish', 'speaks_tamil', 'speaks_lisp', 'speaks_serbian', 'speaks_sanskrit', 'speaks_bengali', 'speaks_catalan', 'speaks_hungarian', 'speaks_irish', 'speaks_urdu', 'speaks_romanian', 'speaks_finnish', 'speaks_bulgarian', 'speaks_ancient greek', 'speaks_yiddish', 'speaks_hawaiian', 'speaks_lithuanian', '